In [7]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [5]:
SEASONS = list(range(2016, 2024))

In [9]:
DATA_DIR = 'data'
STANDINGS_DIR = os.path.join(DATA_DIR, 'standings')
SCORES_DIR = os.path.join(DATA_DIR, 'scores')

In [14]:
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)
        
        try:
            async with async_playwright() as p:
                browser = await p.chromium.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
                
        except PlaywrightTimeout:
            print(f'Timeout error on {url}')
            continue
        else: 
            break
    return html

In [24]:
async def scrape_season(season):
    url = f'https://www.basketball-reference.com/leagues/NBA_{season}_games.html'
    html = await get_html(url, '#content .filter')
    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    href = [l['href'] for l in links]
    standing_pages = [f'https://www.basketball-reference.com{l}' for l in href]
    
    for url in standing_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, '#all_schedule')
        with open(save_path, 'w+') as f:
            f.write(html)

In [25]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Refere

In [43]:
standings_files = os.listdir(STANDINGS_DIR)

In [41]:
async def scrape_game(standings_file):
    with open(standings_file, 'r') as f:
        html = f.read()
        
    soup = BeautifulSoup(html)
    links = soup.find_all('a')
    hrefs = [l.get('href') for l in links]
    box_scores = [l for l in hrefs if l and 'boxscore' in l and '.html' in l ]
    box_scores = [f'https://www.basketball-reference.com{l}' for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split('/')[-1])
        if os.path.exists(save_path):
            continue
        
        html = await get_html(url, '#content')
        if not html:
            continue
        with open(save_path, 'w+') as f:
            f.write(html)

In [45]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)
    
    await scrape_game(filepath)

Nets vs Bucks, October 19, 2021 | Basketball-Reference.com
Warriors vs Lakers, October 19, 2021 | Basketball-Reference.com
Pacers vs Hornets, October 20, 2021 | Basketball-Reference.com
Bulls vs Pistons, October 20, 2021 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202110200NYK.html
Timeout error on https://www.basketball-reference.com/boxscores/202110200NYK.html
Celtics vs Knicks, October 20, 2021 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202110200TOR.html
Wizards vs Raptors, October 20, 2021 | Basketball-Reference.com
Timeout error on https://www.basketball-reference.com/boxscores/202110200MEM.html
Timeout error on https://www.basketball-reference.com/boxscores/202110200MEM.html
Timeout error on https://www.basketball-reference.com/boxscores/202110200MEM.html
Rockets vs Timberwolves, October 20, 2021 | Basketball-Reference.com
76ers vs Pelicans, October 20, 2021 | Basketball-Reference.com
M

CancelledError: 